# IMPORTS

In [23]:
import pandas as pd
from sklearn.model_selection import train_test_split

# CONST

In [31]:
DIRECTORY_PROPERTIES_ARGENTINA: str = '../properati_argentina_2021.csv'
TEST_SIZE: float = 0.2
RANDOM_STATE: int = 42

# MODULE

In [42]:
from pandas import DataFrame

class Propertie:
    def __init__(self, pandas):
        self.pandas = pandas

    def filter_by(self, columns_param: list, conditions_param: list, directory: str) -> DataFrame:
        data_frame = self.pandas.read_csv(directory)
        housing_type = data_frame[columns_param[0]].isin(conditions_param[0])
        is_operation_sale = data_frame[columns_param[1]].isin(conditions_param[1])
        is_property_currency_usd = data_frame[columns_param[2]].isin(conditions_param[2])
        is_place_l2_capital_federal = data_frame[columns_param[3]].isin(conditions_param[3])
        return data_frame[housing_type & is_operation_sale & is_property_currency_usd & is_place_l2_capital_federal]
    
    def get_columns_by_type(self, dataframe: DataFrame) -> (list, list):
        col_numericas = []
        col_categoricas = []
        for columna in dataframe.columns:
            if dataframe[columna].dtype == 'float' or dataframe[columna].dtype == 'int':
                col_numericas.append(columna)
            else:
                col_categoricas.append(columna)
        return col_numericas, col_categoricas
                

    def export_csv_by(self, directory: str, data_type: DataFrame) -> None:
        data_type.to_csv(directory, index=False)
    
properties = Propertie(pd)

# **1. Análisis Exploratorio y Preprocesamiento de Datos**
El primer paso consiste en la selección de los datos que se van a utilizar, se deben filtrar
únicamente los anuncios de propiedades de tipo vivienda ( Casa, PH y Departamento ) ubicados
en Capital Federal cuyo tipo de operación sea venta y su precio se encuentre en dólares (USD).
Se debe separar un conjunto de entrenamiento (80%) y un conjunto de test (20%).

## **PREPRACIÓN DE LOS DATOS**
Crear un nuevo dataset que contemplen los siguientes atributos :
- Tipos de vivienda: Casa, Ph Y Departamentos
- Moneda de transacción: Dólares (USD)
- Ubicación : Capital Federal

In [43]:
columns = ["property_type", "operation", "property_currency", "place_l2"]
conditions = [["Casa", "PH","Departamento"],["Venta"],["USD"],["Capital Federal"]]

### **1.1 Mostrar información del dataset con los filtros requeridos del enunciado**

In [44]:
property_argentina_filtered = properties.filter_by(
    columns,
    conditions,
    directory=DIRECTORY_PROPERTIES_ARGENTINA
)
print("DataFrame filtrado: ", property_argentina_filtered.shape)

DataFrame filtrado:  (94249, 20)


### 1.2 Separar el conjunto de prueba y de entranamiento
- Entrenamiento 80%
- Prueba 20 %

#### Conjunto de Entrenamiento

In [45]:
properties_train, _ = train_test_split(
    property_argentina_filtered, 
    test_size=TEST_SIZE, 
    random_state=RANDOM_STATE
)
print(properties_train.shape)

(75399, 20)


#### Conjunto de prueba

In [46]:
_, properties_test = train_test_split(
    property_argentina_filtered, 
    test_size=TEST_SIZE, 
    random_state=42
)
print(properties_test.shape)

(18850, 20)


Exportamos el conjunto de prueba

In [47]:
directory_export = "properti_argentina_test.csv"
properties.export_csv_by(directory_export, properties_test)

## **ENTENDER EL PROBLEMA**
El objetivo del tp es poder entrenar un modelo que pueda predecir el valor de venta de un inmueble en cualquier barrio de capital federal.
En esta primera parte vamos a limpiar los datos y además entender como los datos pueden influir en la prediction de nuestro modelo o rendimiento de nuestro modelo.

### COMPROBACIONES PLANTEADAS EN EL TP
Las comprobaciones nos daran un entendimiento más profundo sobre los datos. Con los conocimientos de los datos vamos a tener más criterios para poder limpiar los mismos y poder entrenar el modelo.
#### a) Exploración Inicial: analizar cada variable, considerando los siguientes aspectos

### Columnas del Tipo Numericas

In [49]:
col_numericas, _ = properties.get_columns_by_type(properties_train)
col_numericas_pd = pd.DataFrame(col_numericas)
col_numericas_pd

,0
0,latitud
1,longitud
2,place_l6
3,property_rooms
4,property_bedrooms
5,property_surface_total
6,property_surface_covered
7,property_price


### Columnas del tipo categoricas

In [50]:
_, col_categoricas = properties.get_columns_by_type(properties_train)
col_categoricas_pd = pd.DataFrame(col_categoricas)
col_categoricas_pd

,0
0,id
1,start_date
2,end_date
3,created_on
4,place_l2
5,place_l3
6,place_l4
7,place_l5
8,operation
9,property_type
